In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Atrial_Fibrillation"
cohort = "GSE41177"

# Input paths
in_trait_dir = "../../input/GEO/Atrial_Fibrillation"
in_cohort_dir = "../../input/GEO/Atrial_Fibrillation/GSE41177"

# Output paths
out_data_file = "../../output/preprocess/Atrial_Fibrillation/GSE41177.csv"
out_gene_data_file = "../../output/preprocess/Atrial_Fibrillation/gene_data/GSE41177.csv"
out_clinical_data_file = "../../output/preprocess/Atrial_Fibrillation/clinical_data/GSE41177.csv"
json_path = "../../output/preprocess/Atrial_Fibrillation/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Region-specific gene expression profiles in left atria of patients with valvular atrial fibrillation"
!Series_summary	"Of 54,675 expressed sequence tags, microarray analysis revealed that 391 genes were differently expressed (>1.5-fold difference) between LA-PV junction and LAA, including genes related to arrhythmia, cell death, fibrosis, hypertrophy, and inflammation. Microarray and q-PCR produced parallel results in analyzing the expression of particular genes. The expression of paired like homeodomain-2 (PITX2) and its target protein (short stature homeobox-2 [SHOX2]) was greater in LA-PV junction than in LAA, which may contribute to arrhythmogenesis. Five genes related to thrombogenesis were up-regulated in LAA, which may implicate for the preferential thrombus formation in LAA. Genes related to fibrosis were highly expressed in LAA, which was reflected by intense ultrastructural changes in this region"
!Series_overall_design	"Paired LA-PV jun

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import os
import re
from typing import Optional, Dict, Any, Callable

# 1. Gene Expression Data Availability
# Based on the background information, this dataset contains microarray analysis of gene expression profiles
# for patients with atrial fibrillation, comparing LA-PV junction and left atrial appendage.
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# From the Sample Characteristics Dictionary, we can see:
# - All patients have Atrial Fibrillation (constant feature, not useful for analysis)
# - gender: key 1
# - age: key 2

# Define the sample characteristics dictionary from the previous output
sample_chars_dict = {
    0: ['organ: left atrial appendage', 'organ: left atrial junction'], 
    1: ['gender: female', 'gender: male'], 
    2: ['age: 62Y', 'age: 43Y', 'age: 55Y', 'age: 65Y', 'age: 61Y', 'age: 64Y', 'age: 47Y', 'age: 60Y', 'age: 71Y', 'age: 32Y', 'age: 59Y', 'age: 56Y', 'age: 51Y', 'age: 66Y', 'age: 36Y'], 
    3: ['af duration: 0M', 'af duration: 10M', 'af duration: 110M', 'af duration: 15M', 'af duration: >1M', 'af duration: 72M', 'af duration: 102M', 'af duration: 48M', 'af duration: 100M', 'af duration: 73M', 'af duration: 14M', 'af duration: 150M', 'af duration: 78M']
}

# Since all patients have Atrial Fibrillation, it's a constant feature
trait_row = None
age_row = 2
gender_row = 1

# 2.2 Data Type Conversion
def convert_trait(value: str) -> Optional[int]:
    # Since all subjects have Atrial Fibrillation, would return 1 for all
    # But since trait_row is None, this function won't be used
    return 1

def convert_age(value: str) -> Optional[int]:
    # Format is typically "age: XXY" where XX is the age in years
    if value is None:
        return None
    
    match = re.search(r'age:\s*(\d+)Y', value)
    if match:
        return int(match.group(1))
    return None

def convert_gender(value: str) -> Optional[int]:
    # Format is typically "gender: [female/male]"
    if value is None:
        return None
    
    if 'female' in value.lower():
        return 0
    elif 'male' in value.lower():
        return 1
    return None

# 3. Save Metadata for Initial Filtering
# Trait data is not available as a variable (all patients have AF)
is_trait_available = False
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# Since trait_row is None, indicating clinical data is not available in the format we need,
# we skip the Clinical Feature Extraction step


False

### Step 3: Gene Data Extraction

In [4]:
# 1. Get the SOFT and matrix file paths again 
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"Matrix file found: {matrix_file}")

# 2. Use the get_genetic_data function from the library to get the gene_data
try:
    gene_data = get_genetic_data(matrix_file)
    print(f"Gene data shape: {gene_data.shape}")
    
    # 3. Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")


Matrix file found: ../../input/GEO/Atrial_Fibrillation/GSE41177/GSE41177_series_matrix.txt.gz
Gene data shape: (54675, 38)
First 20 gene/probe identifiers:
Index(['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at', '1294_at',
       '1316_at', '1320_at', '1405_i_at', '1431_at', '1438_at', '1487_at',
       '1494_f_at', '1552256_a_at', '1552257_a_at', '1552258_at', '1552261_at',
       '1552263_at', '1552264_a_at', '1552266_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Examining the gene identifiers from the previous step output
# The identifiers (like '1007_s_at', '1053_at', etc.) appear to be probe IDs from an Affymetrix microarray
# These are not direct human gene symbols and will need to be mapped to official gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Analyze the gene annotation dataframe to identify which columns contain the gene identifiers and gene symbols
print("\nGene annotation preview:")
print(f"Columns in gene annotation: {gene_annotation.columns.tolist()}")
print(preview_df(gene_annotation, n=5))

# Check for gene information in the SPOT_ID.1 column which appears to contain gene names
print("\nAnalyzing SPOT_ID.1 column for gene symbols:")
if 'SPOT_ID.1' in gene_annotation.columns:
    # Extract a few sample values
    sample_values = gene_annotation['SPOT_ID.1'].head(3).tolist()
    for i, value in enumerate(sample_values):
        print(f"Sample {i+1} excerpt: {value[:200]}...")  # Print first 200 chars
        # Test the extract_human_gene_symbols function on these values
        symbols = extract_human_gene_symbols(value)
        print(f"  Extracted gene symbols: {symbols}")

# Try to find the probe IDs in the gene annotation
gene_data_id_prefix = gene_data.index[0].split('_')[0]  # Get prefix of first gene ID
print(f"\nGene data ID prefix: {gene_data_id_prefix}")

# Look for columns that might match the gene data IDs
for col in gene_annotation.columns:
    if gene_annotation[col].astype(str).str.contains(gene_data_id_prefix).any():
        print(f"Column '{col}' contains values matching gene data ID pattern")

# Check if there's any column that might contain transcript or gene IDs
print("\nChecking for columns containing transcript or gene related terms:")
for col in gene_annotation.columns:
    if any(term in col.upper() for term in ['GENE', 'TRANSCRIPT', 'SYMBOL', 'NAME', 'DESCRIPTION']):
        print(f"Column '{col}' may contain gene-related information")
        # Show sample values
        print(f"Sample values: {gene_annotation[col].head(3).tolist()}")



Gene annotation preview:
Columns in gene annotation: ['ID', 'GB_ACC', 'SPOT_ID', 'Species Scientific Name', 'Annotation Date', 'Sequence Type', 'Sequence Source', 'Target Description', 'Representative Public ID', 'Gene Title', 'Gene Symbol', 'ENTREZ_GENE_ID', 'RefSeq Transcript ID', 'Gene Ontology Biological Process', 'Gene Ontology Cellular Component', 'Gene Ontology Molecular Function']
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 

Column 'ID' contains values matching gene data ID pattern


Column 'GB_ACC' contains values matching gene data ID pattern


Column 'Target Description' contains values matching gene data ID pattern


Column 'Representative Public ID' contains values matching gene data ID pattern


...

Column 'Gene Symbol' contains values matching gene data ID pattern


Column 'ENTREZ_GENE_ID' contains values matching gene data ID pattern


Column 'RefSeq Transcript ID' contains values matching gene data ID pattern


Column 'Gene Ontology Biological Process' contains values matching gene data ID pattern



Checking for columns containing transcript or gene related terms:
Column 'Species Scientific Name' may contain gene-related information
Sample values: ['Homo sapiens', 'Homo sapiens', 'Homo sapiens']
Column 'Target Description' may contain gene-related information
Sample values: ['U48705 /FEATURE=mRNA /DEFINITION=HSU48705 Human receptor tyrosine kinase DDR gene, complete cds', 'M87338 /FEATURE= /DEFINITION=HUMA1SBU Human replication factor C, 40-kDa subunit (A1) mRNA, complete cds', "X51757 /FEATURE=cds /DEFINITION=HSP70B Human heat-shock protein HSP70B' gene"]
Column 'Gene Title' may contain gene-related information
Sample values: ['discoidin domain receptor tyrosine kinase 1 /// microRNA 4640', 'replication factor C (activator 1) 2, 40kDa', "heat shock 70kDa protein 6 (HSP70B')"]
Column 'Gene Symbol' may contain gene-related information
Sample values: ['DDR1 /// MIR4640', 'RFC2', 'HSPA6']
Column 'ENTREZ_GENE_ID' may contain gene-related information
Sample values: ['780 /// 100616237

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the columns for gene identifiers and gene symbols in the gene annotation data
# From examining the gene annotation data, we can see:
# - 'ID' column contains probe identifiers matching those in gene_data
# - 'Gene Symbol' column contains the gene symbols we need to map to

# 2. Get the gene mapping dataframe
gene_mapping = get_gene_mapping(gene_annotation, 'ID', 'Gene Symbol')
print(f"Gene mapping dataframe shape: {gene_mapping.shape}")
print("Example mapping entries:")
print(gene_mapping.head())

# 3. Apply gene mapping to convert probe-level data to gene expression data
gene_data = apply_gene_mapping(gene_data, gene_mapping)
print(f"Gene expression data shape after mapping: {gene_data.shape}")
print("First 5 genes in mapped data:")
print(gene_data.index[:5])

# Save the gene expression data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to: {out_gene_data_file}")


Gene mapping dataframe shape: (45782, 2)
Example mapping entries:
          ID              Gene
0  1007_s_at  DDR1 /// MIR4640
1    1053_at              RFC2
2     117_at             HSPA6
3     121_at              PAX8
4  1255_g_at            GUCA1A


Gene expression data shape after mapping: (21278, 38)
First 5 genes in mapped data:
Index(['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2M-AS1'], dtype='object', name='Gene')


Gene expression data saved to: ../../output/preprocess/Atrial_Fibrillation/gene_data/GSE41177.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
# Use normalize_gene_symbols_in_index to standardize gene symbols
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape before normalization: {gene_data.shape}")
print(f"Gene data shape after normalization: {normalized_gene_data.shape}")

# Save the normalized gene data to file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene expression data saved to {out_gene_data_file}")

# Load the actual clinical data from the matrix file that was previously obtained in Step 1
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
background_info, clinical_data = get_background_and_clinical_data(matrix_file)

# Get preview of clinical data to understand its structure
print("Original clinical data preview:")
print(clinical_data.head())

# 2. If we have trait data available, proceed with linking
if trait_row is not None:
    # Extract clinical features using the original clinical data
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )

    print(f"Selected clinical data shape: {selected_clinical_df.shape}")
    print("Clinical data preview:")
    print(selected_clinical_df.head())

    # Link the clinical and genetic data
    linked_data = geo_link_clinical_genetic_data(selected_clinical_df, normalized_gene_data)
    print(f"Linked data shape before processing: {linked_data.shape}")
    print("Linked data preview (first 5 rows, 5 columns):")
    print(linked_data.iloc[:5, :5] if not linked_data.empty else "Empty dataframe")

    # 3. Handle missing values
    try:
        linked_data = handle_missing_values(linked_data, trait)
        print(f"Data shape after handling missing values: {linked_data.shape}")
    except Exception as e:
        print(f"Error handling missing values: {e}")
        linked_data = pd.DataFrame()  # Create empty dataframe if error occurs

    # 4. Check for bias in features
    if not linked_data.empty and linked_data.shape[0] > 0:
        is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
        print(f"Data shape after removing biased features: {linked_data.shape}")
    else:
        is_biased = True
        print("Cannot check for bias as dataframe is empty or has no rows after missing value handling")

    # 5. Validate and save cohort information
    note = ""
    if linked_data.empty or linked_data.shape[0] == 0:
        note = "Dataset contains gene expression data related to atrial fibrillation after cardiac surgery, but linking clinical and genetic data failed, possibly due to mismatched sample IDs."
    else:
        note = "Dataset contains gene expression data for atrial fibrillation after cardiac surgery, which is relevant to arrhythmia research."
    
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=True,
        is_trait_available=True,
        is_biased=is_biased,
        df=linked_data,
        note=note
    )

    # 6. Save the linked data if usable
    if is_usable:
        os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
        linked_data.to_csv(out_data_file)
        print(f"Linked data saved to {out_data_file}")
    else:
        print("Dataset is not usable for analysis. No linked data file saved.")
else:
    # If no trait data available, validate with trait_available=False
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=True,
        is_trait_available=False,
        is_biased=True,  # Set to True since we can't use data without trait
        df=pd.DataFrame(),  # Empty DataFrame
        note="Dataset contains gene expression data but lacks proper clinical trait information for arrhythmia analysis."
    )
    
    print("Dataset is not usable for arrhythmia analysis due to lack of clinical trait data. No linked data file saved.")

Gene data shape before normalization: (21278, 38)
Gene data shape after normalization: (19845, 38)


Normalized gene expression data saved to ../../output/preprocess/Atrial_Fibrillation/gene_data/GSE41177.csv
Original clinical data preview:
         !Sample_geo_accession                    GSM1005418  \
0  !Sample_characteristics_ch1  organ: left atrial appendage   
1  !Sample_characteristics_ch1                gender: female   
2  !Sample_characteristics_ch1                      age: 62Y   
3  !Sample_characteristics_ch1               af duration: 0M   

                    GSM1005419                    GSM1005420  \
0  organ: left atrial junction  organ: left atrial appendage   
1               gender: female                  gender: male   
2                     age: 62Y                      age: 43Y   
3              af duration: 0M               af duration: 0M   

                    GSM1005421                    GSM1005422  \
0  organ: left atrial junction  organ: left atrial appendage   
1                 gender: male                  gender: male   
2                     age: